<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/jihoyeo/mobility-simulation-book/blob/main/ko/chapter2.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />구글 코랩에서 실행하기</a>
  </td>
</table>
<br>


# 2. Data Preparation

본 챕터에서는 모빌리티 시뮬레이션을 구축하는데 필요한 기본적인 입력데이터들을 살펴본다. 단순히 데이터를 살펴보는데서 끝나지 않고 간단한 데이터의 전처리 및 시각화의 실습과정도 포함하고 있다.

매우 다양한 형태의 모빌리티 데이터가 존재하며, 이들을 분류하는 기준 역시 분석자의 목표에 따라 여려가지로 구분할 수 있지만 본 책에서는 크게 아래와 같은 형태의 데이터를 살펴볼 것이다.
1. 통행수요 데이터 (Travel Demand)
    - 시뮬레이션을 하기 위한 승객의 수요를 생성하는 기초자료로 활용됨
2. 통행시간 데이터 (Travel Time)
    - 시뮬레이션을 Calibration 하는데 활용
    - e.g., 출퇴근 시간의 교통 정체, 새벽시간대의 빠른 속도를 시뮬레이션에 반영 가능함
3. 도로 네트워크 데이터 (Road Network)
    - 도로 위를 움직이는 차량 구현 (Vehicle Router)
4. 대중교통 네트워크 데이터 (Public Transit Network)
    - 버스, 지하철과 같은 노선과 배차간격이 정해진 모빌리티 시스템을 구현하는데 사용

## 2.1 수도권 생활이동 데이터 분석

### 2.1.1 활용 데이터
- 통행수요는 특정지역에서 출발(Trip Production)하거나, 특정지역으로 도착하는 통행(Trip Attraction) 형태로 존재할 수도 있고, 혹은 더 세분화 한다면 출발지-목적지 단위의 통행수요(Origin-Destination (OD) Travel Demand)로 나타낼 수 있다. 
- 최근 많은 사람들이 스마트폰을 사용하고 따라서 통신사 기지국 기반으로 세밀한 통행패턴이 수집 가능하며, O-D 단위의 통행수요 데이터를 구득하기가 수월해졌다. 따라서 본 교재에서는 O-D 단위의 통행 데이터를 주로 활용하는 실습을 진행할 예정이다. 
- 본 예제에서는 서울시에서 제공하는 수도권 생활이동 데이터를 활용하였다. 
    - https://data.seoul.go.kr/dataVisual/seoul/capitalRegionLivingMigration.do

### 2.1.2 데이터 읽기 및 간단한 전처리

- 간단한 예제를 위해 서울시에서 제공하는 [수도권 생활이동](https://data.seoul.go.kr/dataVisual/seoul/capitalRegionLivingMigration.do) 데이터 중, 2024년 3월 27일 하루치의 데이터만을 예제로 사용하였다. 
- 원활한 실습을 위해 모든 데이터는 Google Drive에 업로드 하였으며, `gdown` 패키지를 사용해 데이터를 다운로드 받은 후 로드하는 방식을 사용하였다. 


In [1]:
import gdown
import pandas as pd
import numpy as np
import os
import topojson as tp # 공간정보를 간소화 해주는 패키지 (분석속도 & 결과물 용량에 영향을 미침)
import geopandas as gpd
import pydeck as pdk

In [2]:
def download_and_read_parquet(file_id, output_path="../data/chp2_od_data_2.parquet"):
    try:
        # Google Drive에서 파일 다운로드
        gdown.download(id=file_id, output=output_path, quiet=False)
        
        # Parquet 파일을 DataFrame으로 읽기
        df = pd.read_parquet(output_path)
        
        # 임시 파일 삭제 (데이터 용량이 매우 큰 경우 사용)
        # os.remove(output_path) # 다운로드 받은 데이터를 삭제하고 싶지 않을 때는 해당 라인을 주석처리
        
        return df
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [3]:
# 파일 불러오기
file_id = "1uJX9MuNX2J6J5SpE-RQy05XPsae6Fg4x" # 구글 드라이브에 업로드 된 파일의 ID
df = download_and_read_parquet(file_id)

Downloading...
From: https://drive.google.com/uc?id=1uJX9MuNX2J6J5SpE-RQy05XPsae6Fg4x
To: e:\book\urban-mobility-simulation\data\chp2_od_data_2.parquet
100%|██████████| 56.6M/56.6M [00:05<00:00, 11.3MB/s]


인터넷 환경으로 인해 파일을 불러오는게 느리다면, github에서 clone 할 때 가지고 온 data 폴더에 있는 파일을 직접 열 수 있다.  
아래 코드를 실행해 보자.  
```code-cell
df = pd.read_parquet("../data/chp2_od_data.parquet")
```

데이터 자동 로딩 함수를 아래와 같이 구현할 수도 있다. 
매번 데이터를 다운로드하는 것은 시간이 오래 걸리므로, 로컬에 파일이 있는지 먼저 확인하고 없을 때만 구글 드라이브에서 다운로드하는 함수를 만들어보자.

이 함수는 다음과 같이 동작합니다:  
1. 먼저 지정된 경로에 파일이 있는지 확인
2. 파일이 있으면 로컬 파일을 불러옴
3. 파일이 없으면 구글 드라이브에서 자동으로 다운로드 후 불러옴

이렇게 하면 처음에는 다운로드가 필요하지만, 그 다음부터는 로컬 파일을 빠르게 불러올 수 있습니다.


In [ ]:
def load_or_download_parquet(file_id, file_path="../data/chp2_od_data_2.parquet"):
    """
    로컬 파일이 존재하면 불러오고, 없으면 구글 드라이브에서 다운로드하는 함수
    
    Args:
        file_id (str): 구글 드라이브 파일 ID
        file_path (str): 로컬 파일 경로
    
    Returns:
        pandas.DataFrame: 불러온 데이터프레임
    """
    if os.path.exists(file_path):
        print(f"로컬 파일이 존재합니다: {file_path}")
        df = pd.read_parquet(file_path)
        print("로컬 파일을 성공적으로 불러왔습니다.")
        return df
    else:
        print(f"로컬 파일이 존재하지 않습니다. 구글 드라이브에서 다운로드합니다.")
        df = download_and_read_parquet(file_id, file_path)
        if df is not None:
            print("구글 드라이브에서 파일을 성공적으로 다운로드하고 불러왔습니다.")
        return df

# 함수를 사용하여 데이터 불러오기
file_id = "1uJX9MuNX2J6J5SpE-RQy05XPsae6Fg4x"
df = load_or_download_parquet(file_id)



In [ ]:
df

- 컬럼의 명세는 아래와 같다. 행정동 단위의 O-D 통행량 및 통행시간 정보를 제공해주고 있다. 더 상세하게는 내/외국인 구분, 국적, 이동목적과 같은 정보도 제공을 해준다.


| 순번 | 영문 컬럼명 | 컬럼 설명 | NULL 여부 | NULL 대체값 | 형식 | 규칙 | 데이터 허용범위 | 비고 |
|------|------------|-----------|-----------|-------------|------|------|-----------------|------|
| 1 | O_ADMDONG_CD | 출발 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 2 | D_ADMDONG_CD | 도착 행정동 | X | - | STRING | - | - | 행안부 8자리 코드체계 |
| 3 | ST_TIME_CD | 출발 시간 | X | - | STRING | - | - | 7-9시/17시-19시는 20분단위, 그 외 1시간 단위 |
| 4 | FNS_TIME_CD | 도착 시간 | X | - | STRING | - | - | 7-9시/17시-19시는 20분단위, 그 외 1시간 단위 |
| 5 | IN_FORN_DIV_NM | 내/외국인 구분 | X | - | STRING | - | - | 내국인, 단기외국인, 장기외국인 |
| 6 | FORN_CITIZ_NM | 국적 | X | - | STRING | - | - | - |
| 7 | MOVE_PURPOSE | 이동 목적 | X | - | STRING | - | - | 1: 출근, 2 : 등교, 3: 귀가, 4: 쇼핑, 5: 관광, 6: 병원, 7: 기타 |
| 8 | MOVE_DIST | 평균 이동 거리(m) | X | - | DOUBLE | - | - | - |
| 9 | MOVE_TIME | 평균 이동 시간(분) | X | - | DOUBLE | - | - | - |
| 10 | CNT | 이동인구 수 | X | - | DOUBLE | (18,2) | - | - |
| 11 | ETL_YMD | 기준 년월 일 | X | - | STRING | yyyyMMdd | 데이터 기준 당일 | - |

### 2.1.3 Basic visualization

- 통행량 데이터의 경우 출발지/도착지에 대한 공간정보 및 통행시간 정보가 포함된 데이터이다. 
- 도시/교통 분야의 데이터는 이처럼 시간과 공간의 정보를 동시에 가지고 있는 '시공간 데이터'인 경우가 많으며 이를 분석하기 위한 기초적인 분석기술 및 핵심 패키지를 이해하는 것이 중요하다.
- 공간데이터를 분석하기 위한 Python 패키지로는 대표적으로 `geopandas`가 있으며, 시각화를 위한 패키지로는 `folium`, `plotly`, `pydeck`이 있다. 
- 위의 키워드로하여 인터넷 검색을 해보면 다양한 튜토리얼 및 예제 코드가 있으므로 시간을 할애해서 공부하는 것을 추천하며, 본 책에서는 상세히 다루지 않는다.
- 아래는 내가 추천하는 시간을 할애해서 공부하면 좋은 자료들이다.
    - [Kaggle 에서 제공해주는 Geospatial Analysis](https://www.kaggle.com/learn/geospatial-analysis)
    - [Pydeck을 활용한 공간데이터 시각화](https://deckgl.readthedocs.io/en/latest/)
    - [Geocomputation with Python](https://py.geocompx.org/08-mapping)


- 수도권 생활이동 데이터의 경우 행정동 단위로 데이터가 존재하기 때문에 공간상에 시각화를 하기 위해서는 행정동의 geometry 정보를 담고 있는 데이터가 추가로 필요하다.
- 본 실습에서는 https://github.com/vuski/admdongkor 에서제공하는 행정동 경계 데이터를 활용하여 간단한 시각화를 수행한다

In [ ]:
def download_and_read_geojson(file_id, output_path="../data/chp2_HangJeongDong_ver20230101.geojson"):
    try:
        # Google Drive에서 파일 다운로드
        gdown.download(id=file_id, output=output_path, quiet=False)
        
        # Parquet 파일을 DataFrame으로 읽기
        df = gpd.read_file(output_path, driver='GeoJSON')
        
        # 임시 파일 삭제 (데이터 용량이 매우 큰 경우 사용)
        # os.remove(output_path) # 다운로드 받은 데이터를 삭제하고 싶지 않을 때는 해당 라인을 주석처리
        
        return df
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [ ]:
#### 행정경계 데이터 불러오기
file_id = "1u8V4h-yUef0g-RJ445wsBgHXpMyUj7ih" # 구글 드라이브에 업로드 된 파일의 ID
gdf_adm = download_and_read_geojson(file_id)
# gdf_adm = gpd.read_file('../data/chp2_HangJeongDong_ver20230101.geojson', driver='GeoJSON')

In [ ]:
#### 서울, 경기, 인천 지역만 추출
gdf_adm = gdf_adm[gdf_adm['sidonm'].isin(['서울특별시', '경기도', '인천광역시'])].reset_index(drop=True)

#### 유동인구와 일치하는 행정동코드 컬럼(`adm_cd2`)을 숫자형태로 변환
gdf_adm['adm_cd2'] = pd.to_numeric(gdf_adm['adm_cd2'])

#### Divide by 100 and create the new column 'ADMDONG_CD'
gdf_adm['ADMDONG_CD'] = gdf_adm['adm_cd2'] / 100

#### Convert the 'ADMDONG_CD' column to integer
gdf_adm['ADMDONG_CD'] = (gdf_adm['ADMDONG_CD']).astype(int)

#### name, sgg 컬럼만 남기기
gdf_adm = gdf_adm[['adm_nm','ADMDONG_CD','geometry']].copy()

In [ ]:
gdf_adm

In [ ]:
gdf_adm.plot()

In [ ]:
#### 행정동 데이터 기하구조 간소화
simplified_geometry = tp.Topology(gdf_adm, toposimplify=.005).to_gdf()

In [ ]:
simplified_geometry.plot()

In [ ]:
# Interactive Map
simplified_geometry.explore(tiles = 'CartoDB positron')

In [ ]:
df

In [ ]:
#### 출발 행정동 및 시간단위로 통행량 aggregation
df_agg = df.groupby(['O_ADMDONG_CD', 'ST_TIME_CD']).agg({'CNT': 'sum'}).reset_index()
df_agg

In [ ]:
# 1. 먼저 pandas의 merge 함수를 사용하여 df_agg를 기준으로 병합
merged_df = df_agg.merge(simplified_geometry, 
                         left_on='O_ADMDONG_CD', 
                         right_on='ADMDONG_CD', 
                         how='inner')

# 2. 병합 결과를 GeoDataFrame으로 변환
vis_gdf = gpd.GeoDataFrame(merged_df, geometry='geometry', crs=simplified_geometry.crs)

In [ ]:
vis_gdf[vis_gdf['ST_TIME_CD'] == 1].explore(column='CNT', 
                                            legend=True, 
                                            cmap='Reds', 
                                            tiles='CartoDB positron',
                                            style_kwds={'fillOpacity': 1, 'weight': 0.5})

## 2.2 스마트카드 데이터 분석

- 대중교통 이용자들의 승하차 정보를 담고 있는 스마트카드 데이터는 도시 교통 패턴을 이해하는 데 핵심적인 자료입니다.
- 본 섹션에서는 서울시 지하철 승하차 데이터를 활용하여 기본적인 분석 및 시각화를 수행합니다.
- 스마트카드 데이터는 정확한 시간과 위치 정보를 제공하므로 교통 패턴 분석에 매우 유용합니다.

### 2.2.1 활용 데이터
- 서울시에서 공개하는 지하철 역별 승하차 인원수 데이터를 활용합니다.
- 데이터는 역별, 시간대별 승차 및 하차 인원수를 포함합니다.
- 본 실습에서는 하루 단위의 데이터를 사용하여 시간대별 승하차 패턴을 분석합니다.

In [ ]:
# 스마트카드 데이터 생성 (실제 데이터가 없어 샘플 데이터 생성)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta

# 서울시 주요 지하철역 리스트 (샘플)
subway_stations = [
    '강남역', '홍대입구역', '명동역', '서울역', '잠실역', 
    '신촌역', '이태원역', '여의도역', '종로3가역', '동대문역사문화공원역',
    '건대입구역', '성신여대입구역', '왕십리역', '구로디지털단지역', '신림역'
]

# 시간대 설정 (05시부터 24시까지)
hours = list(range(5, 25))

# 샘플 스마트카드 데이터 생성
smartcard_data = []
for station in subway_stations:
    for hour in hours:
        # 출퇴근 시간대에 더 많은 승하차 인원 설정
        base_boarding = np.random.randint(50, 200)
        base_alighting = np.random.randint(50, 200)
        
        # 출근시간 (7-9시) 승차 증가
        if hour in [7, 8, 9]:
            boarding = base_boarding * np.random.uniform(2.5, 4.0)
        # 퇴근시간 (18-20시) 하차 증가  
        elif hour in [18, 19, 20]:
            alighting = base_alighting * np.random.uniform(2.5, 4.0)
            boarding = base_boarding * np.random.uniform(1.2, 2.0)
        else:
            boarding = base_boarding
            alighting = base_alighting
        
        smartcard_data.append({
            'station_name': station,
            'hour': hour,
            'boarding': int(boarding),
            'alighting': int(alighting if 'alighting' in locals() else base_alighting)
        })

smartcard_df = pd.DataFrame(smartcard_data)
smartcard_df.head(10)

### 2.2.2 시간대별 승하차 패턴 분석

In [ ]:
# 시간대별 전체 승하차 인원 집계
hourly_total = smartcard_df.groupby('hour')[['boarding', 'alighting']].sum().reset_index()

# 시간대별 승하차 패턴 시각화
fig, ax = plt.subplots(1, 1, figsize=(12, 6))

ax.plot(hourly_total['hour'], hourly_total['boarding'], 
        marker='o', linewidth=2, label='승차 인원', color='blue')
ax.plot(hourly_total['hour'], hourly_total['alighting'], 
        marker='s', linewidth=2, label='하차 인원', color='red')

ax.set_xlabel('시간대')
ax.set_ylabel('승하차 인원수')
ax.set_title('시간대별 지하철 승하차 패턴')
ax.legend()
ax.grid(True, alpha=0.3)
ax.set_xticks(range(5, 25))

plt.tight_layout()
plt.show()

In [ ]:
# 역별 하루 총 승하차 인원 순위
station_total = smartcard_df.groupby('station_name')[['boarding', 'alighting']].sum()
station_total['total'] = station_total['boarding'] + station_total['alighting']
station_ranking = station_total.sort_values('total', ascending=False)

print("역별 하루 총 승하차 인원 순위 (Top 10):")
print(station_ranking.head(10))

### 2.2.3 역별 승하차 패턴 히트맵 시각화

In [ ]:
# 역별-시간대별 승차 인원 피벗 테이블 생성
boarding_pivot = smartcard_df.pivot(index='station_name', columns='hour', values='boarding')

# 히트맵 생성
fig, ax = plt.subplots(1, 1, figsize=(15, 8))
sns.heatmap(boarding_pivot, 
            cmap='YlOrRd', 
            annot=False, 
            fmt='.0f',
            cbar_kws={'label': '승차 인원수'},
            ax=ax)

ax.set_title('역별-시간대별 지하철 승차 인원 히트맵')
ax.set_xlabel('시간대')
ax.set_ylabel('지하철역')
plt.xticks(rotation=0)
plt.yticks(rotation=0)
plt.tight_layout()
plt.show()

### 2.2.4 출퇴근 시간대 집중도 분석

In [ ]:
# 출퇴근 시간대 정의
rush_hour_morning = [7, 8, 9]  # 오전 출근시간
rush_hour_evening = [18, 19, 20]  # 오후 퇴근시간

# 각 역별 출퇴근 시간 집중도 계산
station_analysis = []

for station in subway_stations:
    station_data = smartcard_df[smartcard_df['station_name'] == station]
    
    # 전체 승차 인원
    total_boarding = station_data['boarding'].sum()
    
    # 출근시간 승차 인원
    morning_boarding = station_data[station_data['hour'].isin(rush_hour_morning)]['boarding'].sum()
    
    # 퇴근시간 하차 인원  
    evening_alighting = station_data[station_data['hour'].isin(rush_hour_evening)]['alighting'].sum()
    
    # 집중도 계산
    morning_concentration = morning_boarding / total_boarding * 100 if total_boarding > 0 else 0
    
    station_analysis.append({
        'station_name': station,
        'total_boarding': total_boarding,
        'morning_boarding': morning_boarding,
        'evening_alighting': evening_alighting,
        'morning_concentration': morning_concentration
    })

concentration_df = pd.DataFrame(station_analysis)
concentration_df = concentration_df.sort_values('morning_concentration', ascending=False)

print("출근시간 승차 집중도가 높은 역 순위:")
print(concentration_df[['station_name', 'morning_concentration']].head(10))

## 2.3 Travel Time Data (Taxi Trip Record)

- 지금까지는 통행량 데이터를 살펴보았습니다. 이 장에서는 통행시간 데이터를 살펴봅니다.
- 스마트카드(지하철,버스), 택시 탑승 및 호출 이력, 스마트폰 GPS 데이터 등 다양한 경로를 통해 여러분들의 이동 데이터가 수집되고 있습니다.
- 그 중 택시 데이터는 승차와 하차의 정확한 위치와 시간을 알 수 있다는 점에서 데이터의 신뢰성이 높습니다.
- 다양한 국가에서 이미 표준화된 형태의 택시 승하차 데이터를 공개하고 있습니다.
    - [Ney York City Taxi Trip Duration Data](https://www.kaggle.com/competitions/nyc-taxi-trip-duration/data)
    - [Chicago Taxi Trips data](https://data.cityofchicago.org/Transportation/Taxi-Trips-2013-2023-/wrvz-psew/about_data)
- 본 장에서는 서울의 택시 승하차 이력 데이터를 살펴보겠습니다. 다만, 국내의 경우 택시승하차 데이터가 오픈되어있지 않으므로, 여기선 가상의 데이터를 활용합니다.
    - 본 실습 데이터는 실제 데이터가 아니며, 분석을 위해 랜덤하게 생생된 1일치의 데이터입니다. 
    - 국내도 데이터 개방이 더 활성화 되어, 미국의 주요도시처럼 택시 및 스마트카드 데이터가 공개되는 날이 오기를 희망합니다.

### 2.3.1 데이터 읽기

In [ ]:
def download_and_read_parquet(file_id, output_path="../data/chp2_tx_data_generated.parquet"):
    try:
        # Google Drive에서 파일 다운로드
        gdown.download(id=file_id, output=output_path, quiet=False)
        
        # Parquet 파일을 DataFrame으로 읽기
        df = pd.read_parquet(output_path)
        
        # 임시 파일 삭제 (데이터 용량이 매우 큰 경우 사용)
        # os.remove(output_path) # 다운로드 받은 데이터를 삭제하고 싶지 않을 때는 해당 라인을 주석처리
        
        return df
    except Exception as e:
        print(f"오류 발생: {e}")
        return None

In [ ]:
# 파일 불러오기
file_id = "1uJj-C_7pTkcRk8p5lYah-9Fp660OvS4b" # 구글 드라이브에 업로드 된 파일의 ID
tx_data = download_and_read_parquet(file_id)
# tx_data = pd.read_parquet('../data/chp2_tx_data_generated.parquet')

- 데이터의 시간 형식이 숫자형태로 되어있다. 이렇게 되어있을 경우 보기가 불편하고, 시간단위 연산이 어렵기 때문에 시간 타입의 데이터로 변환해주는 것이 좋다

In [ ]:
# 시간 데이터 형식 변횐
tx_data['RIDE_DTIME'] = pd.to_datetime(tx_data['RIDE_DTIME'], format="%Y%m%d%H%M%S")
tx_data['ALIGHT_DTIME'] = pd.to_datetime(tx_data['ALIGHT_DTIME'], format="%Y%m%d%H%M%S")

# 통행시간 계산
tx_data['TRAVEL_TIME'] = (tx_data['ALIGHT_DTIME'] - tx_data['RIDE_DTIME']) / pd.Timedelta(minutes=1)

In [ ]:
tx_data

### 2.3.2 Basic Visualization

- 통행량 데이터와 마찬가지로 간단한 시각화를 해보자.
- 다음과 같은 오픈소스를 활용한다면 매끄럽게 시각화가 가능하다.
    - https://kepler.gl/
    - 데이터를 csv로 저장한 후, Kepler를 통해 시각화를 해보자.
    - [예시](https://kepler.gl/demo/map?mapUrl=https://dl.dropboxusercontent.com/scl/fi/8k0y58t4rceqvslmvhwyl/keplergl_7sjanmo.json?rlkey=8d6d24vw7l63t2bsch5cd53m9&dl=0)
- 통행시간의 경우 출발지-목적지(O-D) 단위로 통행시간을 시각화 하는 연습도 해보자.
    - O-D 단위의 시각화의 경우 무수히 많은 O-D pairs가 존재하기 때문에 시각적으로 표현하기가 매우 어렵다.
    - 또한, 지금 데이터의 공간단위가 이전에 다뤘던 행정구역과 같이 합산된 공간단위가 아니라, 세밀한 위경도 좌표로 표현되어 있기 때문에 더 어렵다.
    - 위와 같은 문제들을 어떻게 해결할 수 있을까? 

In [ ]:
tx_data.to_csv('../data/tx_data_generated.csv')

## 2.4 Road Network and Geometry

### 2.4.1 Open Street Map 데이터 살펴보기

[오픈스트리트맵 한국](https://osm.kr/usage/)에서는 대한민국의 OSM을 이용하는 상세한 방법에 대한 메뉴얼을 제공해주고 있다. 상세한 내용은 이곳을 참고하면 되며, 본 장에서는 한국의 OSM 데이터를 불러와서 `OSMnx` 패키지로 간략히 분석하고 시각화 하는 작업을 진행한다. 

- OSM을 이용해서 수도권 지역의 도로 데이터를 가져오고, 시각화 하는 작업을 수행한다  
- OSMnx 이용해서 그래프 분석하고, 복잡한 그래프를 간소화 하는 작업을 실습한다

#### [1] Open Street Map 데이터 취득하기 (.osm.pbf 파일)
- 사이트 : https://download.geofabrik.de/asia.html
- South Korea 부분의 .osm.pbf 버튼 클릭
- 여기서 다운로드 받은 .osm.pbf 파일은 본 챕터에서는 사용하지 않지만 향후 Vehicle Router를 만들 때 활용한다. 

In [ ]:
import networkx as nx
import osmnx as ox
import folium

ox.config(use_cache=True, log_console=True)
ox.__version__

#### [2] 내가 원하는 지역 osm 지도를 그래프 형태로 표현
- 본 실습에서는 `OSMnx` 패키지를 사용하여 원하는 도시의 Graph를 Python으로 가져와서 분석하는 실습을 진행한다.

In [ ]:
# get a graph for some city
# www.openstreetmap.org에서 검색 결과가 city-state-country 단위로 나와야 함 

# # 서울특별시 전체 도로 네트워크 불러오기
# G = ox.graph_from_place('서울특별시, 대한민국', network_type='drive')

# osmnx 그래프 생성( osm 지도 다운로드 )
G = ox.graph_from_place('성남시, 경기도, 대한민국', network_type='drive')

`network_type='drive'`는 도로 네트워크(차량이 다닐 수 있는 도로)만을 가져오는 옵션이다. 따라서 보행자 도로, 자전거 도로, 또는 작은 골목 등은 포함되지 않을 수 있다. 전체적인 도로와 교통 네트워크를 모두 포함하려면 다른 network_type을 사용할 수 있다.

OSMnx에서 제공하는 네트워크 타입 옵션들은 다음과 같다.

- 'all': 모든 도로 네트워크를 가져온다 (자동차, 자전거, 보행자 도로 등 포함)
- 'all_private': 모든 도로 네트워크(사유지 도로 포함)
- 'bike': 자전거 네트워크만 가져온다
- 'walk': 보행자 네트워크만 가져온다
- 'drive': 차량이 다니는 도로만 가져온다
- 'drive_service': 서비스 도로(차량 진입 가능하지만 주요 도로는 아님)도 포함

전체 네트워크를 가져오려면 다음과 같이 사용할 수 있다. 

```python
G = ox.graph_from_place('성남시, 경기도, 대한민국', network_type='all')
```

In [ ]:
## 간단한 시각화
# fig, ax = ox.plot_graph(G)
fig, ax = ox.plot_graph(
                            G, 
                            figsize=(12, 12),
                            node_size=5, 
                            edge_linewidth=0.5
                        )

- 하지만 위와 같이 시각화를 한다면, static map이기 때문에 원하는 특정 지역을 세밀하게 관찰하기가 어렵다.  
- Interactive Map을 사용하면 속도는 더 느리지만, 보다 상세히 그래프를 살펴볼 수 있다. 

In [ ]:
# 그래프의 중심점 계산
center_lat, center_lon = ox.geocode('성남시, 경기도, 대한민국')

# folium 맵 생성
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='cartodbpositron')

# 엣지(도로) 추가
for u, v, data in G.edges(data=True):
    locations = [(G.nodes[u]['y'], G.nodes[u]['x']), (G.nodes[v]['y'], G.nodes[v]['x'])]
    
    # 엣지(street) 정보 생성
    edge_info = f"도로명: {data.get('name', 'Unknown')}<br>" \
                f"길이: {data.get('length', 0):.2f} m<br>" \
                f"도로 유형: {data.get('highway', 'Unknown')}"
    
    folium.PolyLine(
        locations=locations,
        weight=2,
        color='blue',
        opacity=0.7,
        tooltip=edge_info
    ).add_to(m)

# 노드(교차점) 추가
for node, data in G.nodes(data=True):
    # 노드에 연결된 엣지 수 계산
    degree = G.degree(node)
    
    # 노드 정보 생성
    node_info = f"Node ID: {node}<br>" \
                f"위도: {data['y']:.6f}<br>" \
                f"경도: {data['x']:.6f}<br>" \
                f"연결된 도로 수: {degree}"
    
    folium.CircleMarker(
        location=(data['y'], data['x']),
        radius=3,
        popup=node_info,
        color='red',
        fill=True,
        fillColor='red',
        tooltip=f"Node ID: {node}"
    ).add_to(m)

# 맵 저장
m.save("../data/Sungnam_road_network_vis.html")

- 한가지 이상한 점이 있다. 모든 Egde가 직선으로표현되어, 실제 Basemap에 있는 도로와 정확하게 매칭되지 않는다
- Open Steet Map의 edge는 도로의 geometry 정보를 포함하고 있으며, 이 정보를 사용하면 실제도로와 유사하게 시각화가 가능하다

In [ ]:
# folium 맵 생성 (CARTODB positron 베이스맵 사용)
m = folium.Map(location=[center_lat, center_lon], zoom_start=12, tiles='cartodbpositron')

# 엣지(도로) 추가
for u, v, data in G.edges(data=True):
    if 'geometry' in data:
        # 실제 도로 지오메트리 사용
        geometry = data['geometry']
        coordinates = list(geometry.coords)
        locations = [(y, x) for x, y in coordinates]
    else:
        # 지오메트리 정보가 없는 경우 직선으로 표시
        locations = [(G.nodes[u]['y'], G.nodes[u]['x']), (G.nodes[v]['y'], G.nodes[v]['x'])]
    
    # 엣지(street) 정보 생성
    edge_info = f"도로명: {data.get('name', 'Unknown')}<br>" \
                f"길이: {data.get('length', 0):.2f} m<br>" \
                f"도로 유형: {data.get('highway', 'Unknown')}"
    
    folium.PolyLine(
        locations=locations,
        weight=2,
        color='blue',
        opacity=0.7,
        tooltip=edge_info
    ).add_to(m)

# 노드(교차점) 추가
for node, data in G.nodes(data=True):
    # 노드에 연결된 엣지 수 계산
    degree = G.degree(node)
    
    # 노드 정보 생성
    node_info = f"Node ID: {node}<br>" \
                f"위도: {data['y']:.6f}<br>" \
                f"경도: {data['x']:.6f}<br>" \
                f"연결된 도로 수: {degree}"
    
    folium.CircleMarker(
        location=(data['y'], data['x']),
        radius=3,
        popup=node_info,
        color='red',
        fill=True,
        fillColor='red',
        fillOpacity=0.7,
        tooltip=f"Node ID: {node}"
    ).add_to(m)

# 맵 저장
m.save("../data/Sungnam_road_network_vis_with_geometry.html")

위 HTML 파일을 열어서 한번 살펴보자. `Folium`은 Leaflet.js 기반으로 HTML을 생성하고 이를 웹페이지로 렌더링한다.  
지도를 그리거나 데이터를 시각화할 때 매번 HTML 파일을 업데이트하거나 렌더링해야 하기 때문에 시간이 더 소요될 수 있습니다.

데이터가 매우 크고, 수가 많을 경우 GPU 가속을 활용하는 시각화 패키지나 툴을 이용하는 것이 도움이 된다.  
베이스 맵의 경우 https://www.mapbox.com/에서 직접 커스터마이즈 하면 된다.

In [ ]:
import pydeck as pdk
import pandas as pd

# 엣지(도로) 데이터 생성
edges = []
for u, v, data in G.edges(data=True):
    if 'geometry' in data:
        geometry = data['geometry']
        coordinates = list(geometry.coords)
        locations = [(y, x) for x, y in coordinates]
    else:
        locations = [(G.nodes[u]['y'], G.nodes[u]['x']), (G.nodes[v]['y'], G.nodes[v]['x'])]
    
    for i in range(len(locations) - 1):
        edges.append({
            'start_lat': locations[i][0],
            'start_lon': locations[i][1],
            'end_lat': locations[i+1][0],
            'end_lon': locations[i+1][1],
            'name': data.get('name', 'Unknown'),
            'length': data.get('length', 0),
            'highway': data.get('highway', 'Unknown')
        })

edges_df = pd.DataFrame(edges)

# 노드(교차점) 데이터 생성
nodes = []
for node, data in G.nodes(data=True):
    nodes.append({
        'lat': data['y'],
        'lon': data['x'],
        'node_id': node,
        'degree': G.degree(node)
    })

nodes_df = pd.DataFrame(nodes)

# Pydeck의 LineLayer와 ScatterplotLayer 설정
edge_layer = pdk.Layer(
    "LineLayer",
    data=edges_df,
    get_source_position='[start_lon, start_lat]',
    get_target_position='[end_lon, end_lat]',
    get_color=[0, 0, 255],
    get_width=2,
    pickable=True,
    auto_highlight=True,
    tooltip={
        "html": "도로명: {name}<br>길이: {length} m<br>도로 유형: {highway}",
        "style": {"color": "white"}
    }
)

node_layer = pdk.Layer(
    "ScatterplotLayer",
    data=nodes_df,
    get_position='[lon, lat]',
    get_fill_color=[255, 0, 0],
    get_radius=20,
    pickable=True,
    auto_highlight=True,
    tooltip={
        "html": "Node ID: {node_id}<br>연결된 도로 수: {degree}",
        "style": {"color": "white"}
    }
)

# Pydeck View 설정
view_state = pdk.ViewState(
    latitude=center_lat,
    longitude=center_lon,
    zoom=12,
    bearing=0,
    pitch=0
)

# Pydeck 맵 렌더링
r = pdk.Deck(
    layers=[edge_layer, node_layer],
    initial_view_state=view_state
)

# 맵 저장
r.to_html("../data/Sungnam_road_network_vis_with_pydeck.html")

#### [3] 여러 네트워크를 한번에 표시

In [ ]:
# 여러 위치에서 네트워크를 생성하기 위한 서울 지역 리스트 설정
places = [
    '성남시, 경기도, 대한민국',  
    '송파구, 서울특별시, 대한민국',  
    '하남시, 경기도, 대한민국',  
]

# retain_all=True를 사용하여 모든 연결되지 않은 서브그래프도 유지 (연속된 네트워크가 아닌 경우에도 유지)
# 여러 장소를 기반으로 차량 도로 네트워크(network_type="drive")를 생성
G = ox.graph_from_place(places, network_type="drive", retain_all=True)

# 생성된 네트워크 시각화 
fig, ax = ox.plot_graph(
    G,                      
    node_size=0,           
    edge_color="#FFFF5C",    
    edge_linewidth=0.25      
)

#### [4]  기초적인 도로네트워크 속성 파악 및 시각화

In [ ]:
# osmnx 그래프 생성( osm 지도 다운로드 )
G = ox.graph_from_place('성남시, 경기도, 대한민국', network_type='drive')

In [ ]:
# 기본 그래프 정보 출력
print("노드 수:", len(G.nodes))
print("엣지 수:", len(G.edges))

`OSMnx` 패키지에 있는 `basic_stats`라는 함수를 사용하면 기본적인 네트워크 통계를 출력해준다. 아래와 같은 정보들이 포함된다. 

- **n: 그래프의 총 노드 수**
- **m: 그래프의 총 엣지 수**
- **k_avg: 평균 노드 차수 (한 노드에 연결된 평균 엣지 수)**
- **edge_length_total: 모든 엣지 길이의 총합 (미터 단위)**
- edge_length_avg: 평균 엣지 길이 (미터 단위)
- **streets_per_node_avg: 노드당 평균 Street 수**
- streets_per_node_counts: 각 Street수별 노드 개수 (예: 3개의 거리가 만나는 교차로의 수)
- streets_per_node_proportions: 각 Street 수별 노드 비율
- **intersection_count: 교차로의 총 개수**
- **street_length_total: 모든 거리의 총 길이 (미터 단위)**
- **street_segment_count: 거리 세그먼트의 총 개수**
- street_length_avg: 평균 Street 길이 (미터 단위)
- circuity_avg: 평균 우회도 (직선 거리 대비 실제 거리의 비율)
- self_loop_proportion: 자기 루프(같은 노드로 시작하고 끝나는 엣지)의 비율

In [ ]:
# 기본 네트워크 통계 계산 및 출력
stats = ox.basic_stats(G)
print("\n기본 네트워크 통계:")
for key, value in stats.items():
    print(f"{key}: {value}")

보다 세부적으로 Node와 Edge들이 어떤 속성을 가지고 있는지 알아보자.  
기본적으로 Graph의 Node와 Edge는 ~~~ 형태의 데이터 포맷을 가지고 있다. Python에서 보다 쉽게 이를 보기 위해서 Dataframe으로 변환하면 좋다. 

Node의 속성:
- Node ID: 노드의 고유 식별자 (일반적으로 OSM의 ID)
- x, y 좌표: 노드의 경도(longitude, x)와 위도(latitude, y)
- 기타 속성: 노드가 속한 구역의 정보 등 다양한 부가 정보가 포함될 수 있습니다.



In [ ]:
# 처음 20개의 노드만 선택
sample_nodes = list(G.nodes(data=True))[:20]

# 각 노드의 특성을 딕셔너리 리스트로 변환
node_data = []
for node, data in sample_nodes:
    node_info = {'node_id': node}  # 노드의 고유 ID 추가
    node_info.update(data)  # 노드의 기타 속성 추가 (x, y 좌표 등)
    node_data.append(node_info)

# DataFrame으로 변환
node_df = pd.DataFrame(node_data)
node_df

In [ ]:
# 모든 node의 데이터를 가져옴
nodes = list(G.nodes(data=True))

# node에서 highway 속성만을 추출하여 리스트 생성
highway_types = []
for node_id, data in nodes:
    if 'highway' in data:
        highway_types.append(data['highway'])  # highway 속성이 있으면 리스트에 추가

# highway 타입별 개수 세기
highway_counts = pd.Series(highway_types).value_counts()
highway_counts

Edge의 속성:
- start_node, end_node: edge가 연결하는 두 노드의 ID
- length: edge의 길이 (단위는 보통 미터)
- highway type: 도로 유형 (예: residential, primary 등)
- geometry: 도로의 곡선 정보 (라인스트링 형태로 저장)
- oneway: 도로가 일방통행인지 여부

In [ ]:
# 처음 5개의 엣지만 선택
sample_edges = list(G.edges(data=True))[:20]

# 각 엣지의 특성을 딕셔너리 리스트로 변환
edge_data = []
for u, v, data in sample_edges:
    edge_info = {'start_node': u, 'end_node': v}
    edge_info.update(data)
    edge_data.append(edge_info)

# DataFrame으로 변환
df = pd.DataFrame(edge_data)
df

OSM에서 도로 타입이 총 몇가지로 구분이 되는지 알아보려면 아래와 같이 할 수 있다.  
매우 많은 도로유형이 있는 것을 알 수 있다. 각각의 상세한 설명 및 기준이 궁금하면 다음을 참고하자. https://wiki.openstreetmap.org/wiki/Key:highway

In [ ]:
# 모든 edge의 데이터를 가져옴
edges = list(G.edges(data=True))

# highway 속성만을 추출하여 리스트 생성
highway_types = []
for u, v, data in edges:
    if 'highway' in data:
        # 일부 엣지의 highway 속성이 리스트일 수 있어 처리
        if isinstance(data['highway'], list):
            highway_types.extend(data['highway'])
        else:
            highway_types.append(data['highway'])

# highway 타입별 개수 세기
highway_counts = pd.Series(highway_types).value_counts()
highway_counts

### 2.4.2 최단거리 분석

- 두 지점까지의 거리를 추출하는 것은 교통 데이터를 분석할 때 매우 빈번히 쓰인다
- 가장 간단한 것은 직선거리를 추출하는 것이지만, 실제 도로위를 움직이는 차량이나 사람의 움직임과는 큰 차이를 보인다.

#### [1] 두 지점간 Network Distance 구하기 실습

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import folium

%matplotlib inline

In [ ]:
# pip install scikit-learn
# pip install folium
# pip install scipy

In [ ]:
# 원하는 지역 그래프 생성
G = ox.graph_from_place('동대문구, 서울, 대한민국', network_type='drive', simplify=False)

# 그래프 시각화
fig, ax = ox.plot_graph(
                            G, 
                            figsize=(8, 8),
                            node_size=1, 
                            edge_linewidth=0.5
                        )

In [ ]:
# Convert graph to GeoDataFrames
nodes, edges = ox.graph_to_gdfs(G)

In [ ]:
nodes

In [ ]:
edges

In [ ]:
# 출발지 도착지 좌표 설정( 좌표와 가장 가까운 노드를 찾아줌 )
orig_node = ox.nearest_nodes(G, 127.0462200, 37.5804100) #청량리역
dest_node = ox.nearest_nodes(G, 127.0114887, 37.5718616) #동대문역

In [ ]:
# 최단 거리 경로 추출
route = nx.shortest_path(G, orig_node, dest_node, weight='length')

# Create a GeoDataFrame for the route
route_gdf = ox.routing.route_to_gdf(G, route)

In [ ]:
# 최단 거리 추출
len = nx.shortest_path_length(G, orig_node, dest_node, weight='length') / 1000
print(round(len, 1), "킬로미터")

In [ ]:
# Create a map centered on the mean coordinates of the route
center_lat = route_gdf.geometry.centroid.y.median()
center_lon = route_gdf.geometry.centroid.x.median()
m = folium.Map(location=[center_lat, center_lon], zoom_start=16, tiles = 'cartodbpositron')

# Add the route to the map
route_gdf.explore(m=m, color='red', width=10)

# Add markers for origin and destination
orig_node_xy = (G.nodes[orig_node]['y'], G.nodes[orig_node]['x'])
dest_node_xy = (G.nodes[dest_node]['y'], G.nodes[dest_node]['x'])

folium.Marker(location=orig_node_xy, popup='출발지', icon=folium.Icon(color='green')).add_to(m)
folium.Marker(location=dest_node_xy, popup='목적지', icon=folium.Icon(color='red')).add_to(m)

# Display the map
m

#### [2] 이동 속도 및 시간 영향을 고려한 경로 산출

`G = ox.add_edge_speeds(G)`  

이 함수는 그래프의 모든 엣지(도로 구간)에 예상 속도를 부여한다. OSMnx는 도로 유형, 속도 제한 등의 OpenStreetMap 데이터를 기반으로 각 도로 구간의 예상 속도를 추정한다. 만약 특정 도로에 대한 속도 정보가 없다면, 도로 유형에 따른 기본값을 사용한다. 이 함수 실행 후, 각 엣지에는 'speed' 속성이 추가된다 (단위: km/h).


`G = ox.add_edge_travel_times(G)`  

이 함수는 앞서 추가된 속도 정보와 도로 길이를 사용하여 각 엣지의 이동 시간을 계산한다.
계산식은 기본적으로 "시간 = 거리 / 속도"이다.
이 함수 실행 후, 각 엣지에는 'travel_time' 속성이 추가된다 (단위: 초).



이 두 함수를 순서대로 실행함으로써, 그래프의 각 도로 구간(엣지)에 대해 다음과 같은 정보를 얻게 된다. 

- 예상 주행 속도 (km/h)  
- 예상 이동 시간 (초)

In [ ]:
# 위의 그래프의 모든 엣지에 속도를 부여
G = ox.add_edge_speeds(G)

# 모든 에지에 대한 이동 시간(초) 계산
G = ox.add_edge_travel_times(G)

In [ ]:
# 도로 유형별 평균 속도/시간 값 보기
edges = ox.graph_to_gdfs(G, nodes=False)
print(edges.columns)

# edges["highway"] = edges["highway"].astype(str)
edges.groupby("highway")[["length", "speed_kph", "travel_time"]].mean().round(1)

In [ ]:
# 도로 별 속도 지정
hwy_speeds = {"residential": 35, "secondary": 50, "tertiary": 60} # residential : 주거지도로 secondary : 지방도 tertiary: 시도·군도·구도
G = ox.add_edge_speeds(G, hwy_speeds=hwy_speeds)
G = ox.add_edge_travel_times(G)

In [ ]:
# 도로 유형별 평균 속도/시간 값 보기
edges = ox.graph_to_gdfs(G, nodes=False)
print(edges.columns)

# edges["highway"] = edges["highway"].astype(str)
edges.speed_kph

In [ ]:
# 이동 거리와 이동 시간을 최소화한 두 가지 경로를 계산합니다

# 그래프 내에서 출발지 목적지 가져오기
orig = list(G)[1]
dest = list(G)[111]
route1 = ox.shortest_path(G, orig, dest, weight="length")
route2 = ox.shortest_path(G, orig, dest, weight="travel_time")

In [ ]:
# 시각화
fig, ax = ox.plot_graph_routes(
    G, routes=[route1, route2], route_colors=["r", "y"], route_linewidth=6, node_size=0
)

In [ ]:
# 두 경로 비교

# route1의 길이 계산 (거리 기준 최적 경로의 총 길이, 단위: 미터)
route1_length = int(sum(ox.routing.route_to_gdf(G, route1, weight="length")["length"]))

# route2의 길이 계산 (시간 기준 최적 경로의 총 길이, 단위: 미터)
route2_length = int(sum(ox.routing.route_to_gdf(G, route2, weight="travel_time")["length"]))

# route1의 소요 시간 계산 (거리 기준 최적 경로의 총 소요 시간, 단위: 초)
route1_time = int(sum(ox.routing.route_to_gdf(G, route1, weight="length")["travel_time"]))

# route2의 소요 시간 계산 (시간 기준 최적 경로의 총 소요 시간, 단위: 초)
route2_time = int(sum(ox.routing.route_to_gdf(G, route2, weight="travel_time")["travel_time"]))

# 두 경로의 길이와 소요 시간을 출력
print("Route 1 is", route1_length, "meters and takes", route1_time, "seconds.")
print("Route 2 is", route2_length, "meters and takes", route2_time, "seconds.")

#### [3] 특정 노드에서 지정한 거리/시간에 도달할 수 있는 노드 분석( 버퍼 )

OSMnx 패키지를 사용하면 도로 네트워크에서 특정 시작점으로부터 일정 거리 또는 시간 내에 도달할 수 있는 영역을 식별하는 데도 활용할 수 있다. 이러한 방법은 주로, 어떤 시설이 주변에 얼마나 영향을 미치는지 분석하거나 접근성을 분석하는데 주로 쓰인다. 아래 코드는 강남구의 도로 네트워크에서 특정 노드를 중심으로 1000m 반경 내의 서브그래프를 추출하고 시각화하는 예제이다.

In [ ]:
# 도로 네트워크 불러오기
G = ox.graph_from_place('Gangnam, Seoul, South Korea', network_type='drive')

# 특정 노드를 선택 (예: 첫 번째 노드를 선택)
center_node = list(G.nodes)[0]

# 특정 노드를 중심으로 반경 500m 내의 서브그래프를 ego_graph로 추출
radius = 1000  # 반경 (미터 단위)
ego_G = nx.ego_graph(G, center_node, radius=radius, distance='length')

In [ ]:
# 서브그래프를 시각화
fig, ax = ox.plot_graph(ego_G, node_color='yellow', node_size=10, edge_linewidth=0.5, show=False)

# 중심 노드의 좌표 얻기
center_node_coords = (G.nodes[center_node]['y'], G.nodes[center_node]['x'])

# 중심 노드를 붉은색으로 표시
ax.scatter(center_node_coords[1], center_node_coords[0], c='red', s=100, zorder=5)

plt.tight_layout()
plt.show()

### 2.4.3 Graph 간소화
그래프 간소화는 복잡한 네트워크 데이터를 보다 효율적으로 처리하고 분석하기 위해 필요하다. 복잡한 도로망이나 경로 데이터를 단순화함으로써 계산 시간을 줄이고, 시각화나 경로 탐색 등의 작업을 용이하게 할 수 있다. 우리가 계속 사용해왔던 `graph_from_place` 함수는 기본적으로 `simplify=True` 인자를 사용하고 있다. 이는 OpenStreetMap에서 가져온 원시 그래프 데이터를 자동으로 간소화하여 반환한다는 의미이다.

`simplify_graph` 함수는 불필요한 노드와 엣지를 제거하여 그래프를 단순화한다. 교차로나 말단 노드가 아닌 중간 노드들을 합침으로써 도로를 직선화하고, 그래프의 복잡성을 줄인다. 이를 통해 네트워크 분석의 효율성을 높일 수 있다.

`consolidate_intersections` 함수는 일정 거리 이내에 있는 교차로 노드들을 하나로 통합한다. 이는 현실 세계에서 매우 근접해 있는 교차로를 단일 지점으로 처리하여 분석의 정확성을 높이고, 그래프의 구조를 더욱 간소화한다. 하지만 해당 방법은 굉장히 극단적인 방법일 수 있다. 교차로를 합친다는 것은 도로 구조의 큰 변화를 야기하므로 사용에 주의할 필요가 있다. 분석하는 공간범위가 매우 커, `simplify_graph`만으로는 분석이 어려운 경우 사용을 검토해볼 수 있겠다. 

상세한 설명은 https://osmnx.readthedocs.io/en/stable/user-reference.html#osmnx.simplification.simplify_graph 을 참고

#### [1] OSMnx에서 제공하는 그래프를 간소화 하는 두가지 함수

In [ ]:
# Create the raw graph
G_raw = ox.graph_from_place('성남시, 경기도, 대한민국', network_type='drive', simplify=False)

In [ ]:
# Simplify the graph
G_simple = ox.simplify_graph(G_raw)

# Consolidate intersections
G_consolidated = ox.consolidate_intersections(G_simple, rebuild_graph=True, tolerance=0.00057, dead_ends=False) # 약 50m

In [ ]:
print(G_raw)
print(G_simple)
print(G_consolidated)

In [ ]:
# consolidated 그래프 시각화
fig, ax = ox.plot_graph(
                            G_simple, 
                            figsize=(8, 8),
                            node_size=1, 
                            edge_linewidth=0.5
                        )

In [ ]:
# consolidated 그래프 시각화
fig, ax = ox.plot_graph(
                            G_consolidated, 
                            figsize=(8, 8),
                            node_size=1, 
                            edge_linewidth=0.5
                        )

#### [2] `simplify_graph` 자세히 살펴보기

In [ ]:
# 중심 위치 지점(위도, 경도) 정의 ( 서울 시청 )
location_point = (37.5665, 126.9780)

# 중심 위치로부터 500미터 거리 내에서 도로 네트워크 생성
# simplify=False로 설정하여 네트워크를 아직 단순화하지 않음
G = ox.graph_from_point(location_point, dist=500, simplify=False)

# 단순화되지 않은 네트워크 그래프를 플로팅. 노드는 빨간색(r)으로 표시됨
fig, ax = ox.plot_graph(G, node_color="r")

In [ ]:
# 네트워크가 단순화될 때 제거될 노드를 강조
# 끝점인 노드는 빨간색(r)으로 남고, 끝점이 아닌 노드는 노란색(y)으로 표시됨
nc = ["r" if ox.simplification._is_endpoint(G, node, None) else "y" for node in G.nodes()]
fig, ax = ox.plot_graph(G, node_color=nc)

In [ ]:
# 네트워크를 단순화하여 불필요한 중간 노드를 병합
# 위의 노란색 점 제거
G2 = ox.simplify_graph(G)

# 단순화된 그래프에서 자기 루프(self-loop) 엣지와 연결된 노드를 강조
# 자기 루프 엣지는 노드가 자기 자신과 연결되는 엣지임. 이런 노드는 빨간색(r)으로,
# 나머지 노드는 노란색(y)으로 표시됨
loops = [edge[0] for edge in nx.selfloop_edges(G2)]
nc = ["r" if node in loops else "y" for node in G2.nodes()]
fig, ax = ox.plot_graph(G2, node_color=nc)

## 2.5 GTFS (General Transit Feed Specification)

- 대중교통 네트워크 분석을 위한 데이터
- 향후 업데이트 예정
- 금 학기에서는 다루지 않음

## 2.6 Exercise

GPT와 같은 AI 툴을 적극 활용해 문제를 풀어봅시다. 다만, 출력된 코드를 보고 이해하고, 코드가 실행되지 않는다면 어디서 실행되지 않는지 파악하고 올바르게 수정할 수 있어야 합니다.

### 2.6.1 통행 수요 데이터 분석 및 시각화

> 2.1 Section에서 불러온 2024년 3월 27일 통행 데이터를 가지고 아래와 같은 분석을 수행해보자.

1. 시간에 따른 통행량을 그래프로 시각화 해보자. 몇시에 통행이 가장 많으며, 그 양은 얼마인가?  
2. 공간에 따른 통행량의 분포를 읍면동 단위로 시각화 해보자. 어느 지역에서 출발 통행량(Trip Production)이 가장 많으며 어느 지역에서 도착 통행량 (Trip Attraction)이 가장 많은가?  
3. 통행의 시공간적 분포를 함께 분석해보자. 가장 효과적인 시각화 방안은 무엇일까?  
4. 출발통행, 도착통행이 아닌, 출발지-목적지간의 통행을 시각화 해보자. O-D pairs의 수가 지나치게 많다면 이를 효과적으로 시각화 할 수 있을까? 어떤 패키지나 소프트웨어를 활용하면 좋을지 고민해보자. 

### 2.6.2 스마트카드 데이터 분석 및 시각화

> 2.2 Section에서 구현한 스마트카드 데이터 분석 코드를 참고하여 아래와 같은 추가 분석을 수행해보자.

1. 요일별 승하차 패턴의 차이를 분석해보자. 주중과 주말의 패턴이 어떻게 다른가?
2. 역별 승하차 불균형을 분석해보자. 주로 승차가 많은 역과 하차가 많은 역을 구분할 수 있는가?
3. 시간대별 네트워크 부하를 계산해보자. 가장 혼잡한 시간대는 언제인가?

### 2.6.3 통행 시간 데이터 분석 및 시각화

> 2.3 Section에서 불러온 가상의 택시 통행 데이터를 가지고 아래와 같은 분석을 수행해보자.

1. 택시 통행 데이터의 각 행에, 출발지역의 행정동 코드 & 도착지역의 행정동 코드를 매핑해봅시다. 이후 아래문제를 풀어보세요. `geopandas` 패키지에 있는  `sjoin` 함수를 사용하면 됩니다. 
2. 청담동에서 출발해서 연희동 도착하는 택시 통행만을 추출하여, 출발 시간대별(1시간단위로 합산) 통행시간의 변화를 Line Graph로 그려봅시다. 언제 가장 시간이 오래 걸리나요?  

### 2.6.4 도로네트워크 분석

> 2.4 Section에서 실습한 OSM 데이터를 분석하는 코드를 참고하면서 아래 문제를 풀어보자.

1. 가천대학교 AI공학관에서 가천대역까지 가는 최단경로를 뽑아보고 지도상에 시각화 해봅시다. 해당 위치의 좌표는 어떻게 알 수 있을까요? ([구글지도](https://www.google.com/maps)를 사용해봅시다)
2. 가천대역을 중심으로 하여 도보로 15분 내에 도달 가능한 지역을 지도상에 나타내봅시다. 사람의 통행속도는 5km/h로 가정합니다. 